### 载入训练好的llava架构的模型，在数据集`liuhaotian/LLaVA-CC3M-Pretrain-595K`上提取image_embeds和text_embeds

In [1]:
import transformers
from transformers import (
    LlavaForConditionalGeneration,
    LlavaProcessor,
    Trainer,
    TrainingArguments,
    AutoProcessor
)

import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

from PIL import Image
from peft import PeftModel

import logging
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Sequence

import evaluate

# custom package

# 导入读取数据集和处理数据集为向量的工具类
from show_llava.data import LlavaDataset, TrainLLavaModelCollator
from show_llava.util import print_trainable_parameters
# 导入计算id, idcor的package
from utils import (
    intrinsic_dimension, metrics, utils
)

In [2]:
# 指定gpu
device = "cuda:0"
# 初始化日志
logger = logging.getLogger(__name__)

#### 加载模型及processor所需的工具类及工具函数

In [3]:
# 指定要训练的模型路径及训练参数工具类
@dataclass
class ModelArguments:
    # 基础模型路径（ lora 微调基于的初始模型）
    model_name_or_path: Optional[str] = field(default="./qwen2.5_3B_Instruct_clipvL14_model/model001")
    # lora 微调结果路径
    lora_name_or_path: Optional[str] = field(default="./output_model_lora_show/[epoch4-5]qwen2.5_3B_Instruct_clipvL14")

In [4]:
def load_model_processor(modelargs: ModelArguments):
    # 读取模型
    model = LlavaForConditionalGeneration.from_pretrained(
        modelargs.model_name_or_path,
        torch_dtype=torch.bfloat16,
        low_cpu_mem_usage=True,
        device_map=device
    )
    # 合并lora微调模型
    model = PeftModel.from_pretrained(model, modelargs.lora_name_or_path)
    # 加载processor处理器
    processor = LlavaProcessor.from_pretrained(modelargs.model_name_or_path)
    return model, processor

#### 加载数据集所需的工具类和工具函数

In [5]:
# 指定数据集路径工具类
@dataclass
class DataArguments:
    data_path: str = field(
        default=None, metadata={"help": "Path to the training data."}
    )
    # source_length: int = field(default=128)
    # target_length: int = field(default=512)

In [6]:
def load_dataset_collator(processor, dataargs: DataArguments):

    llava_dataset = LlavaDataset(
        dataargs.data_path  # "data/liuhaotian/LLaVA-CC3M-Pretrain-595K"
    )
    data_collator = TrainLLavaModelCollator(processor, -100)

    return llava_dataset, data_collator

#### 设置`model_args`和`data_args`参数

In [7]:
model_args: ModelArguments = ModelArguments(
    model_name_or_path="./qwen2.5_3B_Instruct_clipvL14_model/model001",
    lora_name_or_path="./output_model_lora_show/[epoch4-5]qwen2.5_3B_Instruct_clipvL14"
)

data_args: DataArguments = DataArguments(
    data_path="/home/lsy/shared_data/liuhaotian/LLaVA-CC3M-Pretrain-595K"
)

model_args, data_args

(ModelArguments(model_name_or_path='./qwen2.5_3B_Instruct_clipvL14_model/model001', lora_name_or_path='./output_model_lora_show/[epoch4-5]qwen2.5_3B_Instruct_clipvL14'),
 DataArguments(data_path='/home/lsy/shared_data/liuhaotian/LLaVA-CC3M-Pretrain-595K'))

#### 载入要评测的模型和对应的`processor`，使用`lora`加载的方式

In [8]:
model, processor = load_model_processor(model_args)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

#### 设置并加载读取数据相关的工具变量

In [10]:
eval_dataset, data_collator = load_dataset_collator(processor, data_args)
dataloader_params = {
    "batch_size": 8,
    "collate_fn": data_collator,
    "num_workers": 4,
    "pin_memory": True,
    "persistent_workers": 4,
    "shuffle": False
}
eval_dataloader = DataLoader(eval_dataset, **dataloader_params)

#### eval循环主体

In [12]:
model.eval()
with torch.no_grad():
    all_text_embeds = []
    all_image_embeds = []
    for steps, inputs in tqdm(enumerate(eval_dataloader)):
        print(inputs)

#### evaluate函数主体及创建Trainer（==转为python脚本的封装==）

In [ ]:
def evaluate():
     # 将命令行参数解析成 dataclass 对象
    parser = transformers.HfArgumentParser(
        (ModelArguments, DataArguments, TrainingArguments)
    )
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()
    model, processor = load_model_processor(model_args)
    eval_dataset, data_collator = load_dataset_collator(processor, data_args)
    

    
